# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
import os

filepath = "../api_keys"
# Import API key
from filepath import open_weather_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy


# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

ModuleNotFoundError: No module named 'filepath'

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
#define base and query URL
units = "imperial"
url = "http://api.openweathermap.org/data/2.5/weather?"
query_url = f"{url}appid={open_weather_key}&q={cities[0]}&units={units}"

#get response
response = requests.get(query_url).json()

#print response and analyze format
response


In [ ]:
#create summary dict
data = {"City": [],
       "Cloudiness": [],
       "Country": [],
       "Date": [],
       "Humidity": [],
       "Lat": [],
       "Lng": [],
       "Max Temp": [],
       "Wind Speed": []}

#set counts for print statment in loop
number = 0
sets = 1

#loop through cities in list
for item in cities:
    try:
        #redefine query_url and response for each city in the loop
        query_url = f"{url}appid={open_weather_key}&q={item}&units={units}"
        response = requests.get(query_url).json()
        #update print statement count and print
        number = number + 1
        print(f"Processing Record {number} of Set {sets} | {item}")
        #append summary dict with appropriate data
        data["City"].append(response['name'])
        data["Cloudiness"].append(response['clouds']['all'])
        data["Country"].append(response['sys']['country'])
        data["Date"].append(response['dt'])
        data["Humidity"].append(response['main']['humidity'])
        data["Lat"].append(response['coord']['lat'])
        data["Lng"].append(response['coord']['lon'])
        data["Max Temp"].append(response['main']['temp_max'])
        data["Wind Speed"].append(response['wind']['speed'])
    #create exception
    except:
        print("City not found. Skipping ...")
    #reset set counts after 50 records
    if number == 50:
        sets = sets+1
        number = 0
    #limit API requests
    time.sleep(1)

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
#convert summary dict to dataframe and display
city_data_df = pd.DataFrame(data)
city_data_df

In [ ]:
#export dataframe to CSV file
city_data_df.to_csv("Output/city_data.csv",
                  encoding="utf-8", index=False, header=True)

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
#defines variable to plot
max_temp = city_data_df["Max Temp"]
latitude = city_data_df["Lat"]
#set date
city_data_df['Date'] = pd.to_datetime(city_data_df['Date'],unit='s')
date = city_data_df["Date"][0]
#create plot
plt.scatter(latitude, max_temp, s=40, c="slateblue", edgecolors = "black", alpha = .75)
#set labels, titles, and other styling
plt.title(f"City Latitude vs. Temperature ({date})")
plt.xlabel("Latitude")
plt.ylabel("Max Temp (F)")
plt.grid(b=None, which='major', axis='both')

#### Latitude vs. Humidity Plot

In [ ]:
#defines variable to plot
humidity = city_data_df["Humidity"]
#create plot
plt.scatter(latitude, humidity, s=40, c="darkseagreen", edgecolors = "black", alpha = .75)
#set labels, titles, and other styling
plt.title(f"City Latitude vs. Humidity ({date})")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.grid(b=None, which='major', axis='both')

#### Latitude vs. Cloudiness Plot

In [ ]:
#defines variable to plot
cloudiness = city_data_df["Cloudiness"]
#create plot
plt.scatter(latitude, cloudiness, s=40, c="cadetblue", edgecolors = "black", alpha = .75)
#set labels, titles, and other styling
plt.title(f"City Latitude vs. Cloudiness ({date})")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.grid(b=None, which='major', axis='both')

#### Latitude vs. Wind Speed Plot

In [ ]:
#defines variable to plot
wind_speed = city_data_df["Wind Speed"]
#create plot
plt.scatter(latitude, wind_speed, s=40, c="indianred", edgecolors = "black", alpha = .75)
#set labels, titles, and other styling
plt.title(f"City Latitude vs. Wind Speed ({date})")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.grid(b=None, which='major', axis='both')

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots
'''
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_axis, y_axis)
regress_values = x_axis * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
'''

def lin_reg(x_axis, y_axis):
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
    regress_values = x_axis * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.plot(x_axis, regress_values, "black")
    plt.show
    return print(f"The value of r-squares is {rvalue**2}.") 
    return print("The line equation: {line_eq}.")



In [ ]:
# Create Northern and Southern Hemisphere DataFrames
northern_df = city_data_df.loc[city_data_df["Lat"] >= 0]
southern_df = city_data_df.loc[city_data_df["Lat"] < 0]

northern_df

In [ ]:
southern_df

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
#defines variable to plot
n_max_temp = northern_df["Max Temp"]
n_latitude = northern_df["Lat"]

#create plot
plt.scatter(n_latitude, n_max_temp, s=40, c="slateblue", edgecolors = "black", alpha = .75)
#set labels, titles, and other styling
plt.title(f"City Latitude vs. Temperature in Northern Hemisphere({date})")
plt.xlabel("Latitude")
plt.ylabel("Max Temp (F)")
plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
#defines variable to plot
s_max_temp = southern_df["Max Temp"]
s_latitude = southern_df["Lat"]

#create plot
plt.scatter(s_latitude, s_max_temp, s=40, c="slateblue", edgecolors = "black", alpha = .75)
#set labels, titles, and other styling
plt.title(f"City Latitude vs. Temperature in Southern Hemisphere({date})")
plt.xlabel("Latitude")
plt.ylabel("Max Temp (F)")
plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#defines variable to plot
n_humidity = northern_df["Humidity"]
#create plot
plt.scatter(n_latitude, n_humidity, s=40, c="darkseagreen", edgecolors = "black", alpha = .75)
#set labels, titles, and other styling
plt.title(f"City Latitude vs. Humidity in Northern Hemisphere({date})")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#defines variable to plot
s_humidity = southern_df["Humidity"]
#create plot
plt.scatter(s_latitude, s_humidity, s=40, c="darkseagreen", edgecolors = "black", alpha = .75)
#set labels, titles, and other styling
plt.title(f"City Latitude vs. Humidity in Southern Hemisphere({date})")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
#defines variable to plot
n_cloudiness = northern_df["Cloudiness"]
#create plot
plt.scatter(n_latitude, n_cloudiness, s=40, c="cadetblue", edgecolors = "black", alpha = .75)
#set labels, titles, and other styling
plt.title(f"City Latitude vs. Cloudiness in Northern Hemisphere({date})")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
#defines variable to plot
s_cloudiness = southern_df["Cloudiness"]
#create plot
plt.scatter(s_latitude, s_cloudiness, s=40, c="cadetblue", edgecolors = "black", alpha = .75)
#set labels, titles, and other styling
plt.title(f"City Latitude vs. Cloudiness in Southern Hemisphere({date})")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#defines variable to plot
n_wind_speed = northern_df["Wind Speed"]
#create plot
plt.scatter(n_latitude, n_wind_speed, s=40, c="indianred", edgecolors = "black", alpha = .75)
#set labels, titles, and other styling
plt.title(f"City Latitude vs. Wind Speed in Northern Hemisphere ({date})")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#defines variable to plot
s_wind_speed = southern_df["Wind Speed"]
#create plot
plt.scatter(s_latitude, s_wind_speed, s=40, c="indianred", edgecolors = "black", alpha = .75)
#set labels, titles, and other styling
plt.title(f"City Latitude vs. Wind Speed in Southern Hemisphere ({date})")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.show()
#linear regression
lin_reg(s_latitude, s_wind_speed)